In [201]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t, anisotropic = []):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        self.anisotropic = anisotropic
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_anisotropic(self, energy_group_idx):
        return self.anisotropic[energy_group_idx]
        
        


# In[3]:


energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[4]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[5]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = 0.
        for xs in self.xs:
            delta_xs = max(delta_xs, max(xs.sig_t))
        self.delta_xs = delta_xs           
            
    def get_delta_xs(self):
        
        return self.delta_xs
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_z = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_x = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    
    if (particle.direction.tetta_z * particle.direction.tetta_z + 
    particle.direction.tetta_y * particle.direction.tetta_y + 
    particle.direction.tetta_x * particle.direction.tetta_x > 1.01):
        print("dir_x * dir_x + dir_y * dir_y + dir_z * dir_z")
        print(dir_x * dir_x + dir_y * dir_y + dir_z * dir_z)
    

In [202]:

#r = 1.0d0
 #                  r1 = 1.0d0
  #                 do while(r**2+r1**2 > 1.0d0)
   #                   call random_number(r)
    #                  r = 2.d0*r - 1.0d0
     #                 call random_number(r1)
      #                r1 = 2.d0*r1 - 1.0d0
       #            end do
                   
        #           dir_x_tmp = dir_x*mu + dsqrt(1.0d0 - mu**2)*(r*dir_x*dir_z - r1*dir_y)/dsqrt(r**2+r1**2)/dsqrt(1.d0-dir_z**2)
         #          dir_y_tmp = dir_y*mu + dsqrt(1.0d0 - mu**2)*(r*dir_y*dir_z + r1*dir_x)/dsqrt(r**2+r1**2)/dsqrt(1.d0-dir_z**2)
          #         dir_z_tmp = dir_z*mu - dsqrt(1.0d0 - mu**2)*r*dsqrt(1.d0-dir_z**2)/dsqrt(r**2+r1**2)
           #        dir_x = dir_x_tmp
            #       dir_y = dir_y_tmp
             #      dir_z = dir_z_tmp

In [203]:
def calculate_mu_4(anisotropic_coeffs):
    random.seed(time.time())
    
    a = (3/2 - 3/2 * (anisotropic_coeffs[1] / anisotropic_coeffs[0]))
    
    rand = random.uniform(0, 1)
    old_mu = 2. * math.sqrt(rand) - 1.
    f_mu = (1 + old_mu)/2.
    delta = 1
        
    mu = a * f_mu + (1 - a) * delta       
        
    return mu 

In [216]:
def calculate_mu_dirac(anisotropic_coeffs):
    random.seed(time.time())
    
    rand = random.uniform(0, 1)
    
    mu = 0
    if rand <= (1 - anisotropic_coeffs[1] / anisotropic_coeffs[0]):
        fi = random.uniform(0, 1)
        mu = 2 * fi - 1
    else:
        mu = 1
        
    return mu 

In [217]:
def calculate_mu(anisotropic_coeffs):
    
    random.seed(time.time())
    
    b = 3. * anisotropic_coeffs[1] / anisotropic_coeffs[0]
    gamma =  random.uniform(0, 1)
    sqr = (1/2 - b)**2 + 2 * b * gamma
    

    result_mu1 = (-1/2 + math.sqrt(sqr))/b
    result_mu2 = (-1/2 - math.sqrt(sqr))/b
    if abs(result_mu1) < 1:
        return result_mu1
    
    if abs(result_mu2) < 1:
        return result_mu2
    
    
    return anisotropic_coeffs[1] / anisotropic_coeffs[0]

In [218]:
def get_anisotropic_direction(particle, mu):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
    
    dir_x = particle.direction.tetta_x
    dir_y = particle.direction.tetta_y
    dir_z = particle.direction.tetta_z
    rr1 = math.sqrt(r**2 + r1**2)
    rmr1 = math.sqrt(1. - dir_z**2)
    mu_1 = math.sqrt(1. - mu**2)
    
    dir_x_tmp = dir_x * mu + mu_1 * (r * dir_x * dir_z - r1 * dir_y) / rr1 / rmr1
    dir_y_tmp = dir_y * mu + mu_1 * (r * dir_y * dir_z + r1 * dir_x) / rr1 / rmr1
    dir_z_tmp = dir_z * mu - mu_1 * r* rmr1 /rr1
    particle.direction.tetta_x = dir_x_tmp
    particle.direction.tetta_y = dir_y_tmp
    particle.direction.tetta_z = dir_z_tmp
  #  dir_y = dir_y_tmp
   # dir_z = dir_z_tmp
    
    

In [219]:
def get_anisotropic_direction_2(particle, mu):
    
    
    dir_x = particle.direction.tetta_x
    dir_y = particle.direction.tetta_y
    dir_z = particle.direction.tetta_z
    
    b = math.sqrt(1 - mu * mu)
    ro = math.sqrt(1 - dir_z * dir_z)
    phi = random.uniform(-math.pi, math.pi)

    c = math.cos(phi)
    d = math.sin(phi)
    a = mu
    
    dir_x_tmp = (b * c * dir_x * dir_z)/ro  - (b * d * dir_y)/ro + a * dir_x    
    dir_y_tmp = (b * c * dir_y * dir_z)/ro  + (b * d * dir_x)/ro + a * dir_y  
    dir_z_tmp = -b * c * ro  + a * dir_z
    
    particle.direction.tetta_x = dir_x_tmp
    particle.direction.tetta_y = dir_y_tmp
    particle.direction.tetta_z = dir_z_tmp
    
    
    

In [237]:
def get_anisotropic_direction_3(particle, mu):
    
    
    dir_x = particle.direction.tetta_x
    dir_y = particle.direction.tetta_y
    dir_z = particle.direction.tetta_z
    a = mu
    b = math.sqrt(1. - mu * mu)
    
    r = 1.5
    r1 = 1.5
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    cos_phi = r1/ math.sqrt(r *r + r1 * r1)
    sin_phi = r/ math.sqrt(r *r + r1 * r1)
    
    c = cos_phi
    d = sin_phi
    
    
    ro = math.sqrt(1. - dir_z * dir_z)
    
    dir_x_tmp = (b * c * dir_x * dir_z) / ro - (b * d * dir_y) / ro + a * dir_x
    dir_y_tmp = (b * c * dir_y * dir_z) / ro + (b * d * dir_x) / ro + a * dir_y
    dir_z_tmp = -b * c * math.sqrt(1. - dir_z * dir_z) + a * dir_z
    

    particle.direction.tetta_x = dir_x_tmp
    particle.direction.tetta_y = dir_y_tmp
    particle.direction.tetta_z = dir_z_tmp
    
  #  print
    
    

In [238]:
def set_random_direction_anisotropic(particle, xs, border):
    
    energy_group_idx = particle.get_energy_group()
    anisotropic_coeffs = xs.get_anisotropic(energy_group_idx)   
    random.seed(time.time())
    mu = calculate_mu(anisotropic_coeffs)


    dir_x_before = particle.direction.tetta_x
    dir_y_before = particle.direction.tetta_y
    dir_z_before = particle.direction.tetta_z

    
    get_anisotropic_direction_3(particle, mu)
        

    dir_x_after = particle.direction.tetta_x
    dir_y_after = particle.direction.tetta_y
    dir_z_after = particle.direction.tetta_z
    
    dot = dir_x_before * dir_x_after + dir_y_before * dir_y_after + dir_z_before * dir_z_after
    
    x_len = math.sqrt(dir_x_before * dir_x_before + dir_y_before * dir_y_before + dir_z_before * dir_z_before)
    y_len = math.sqrt(dir_x_after * dir_x_after + dir_y_after * dir_y_after + dir_z_after * dir_z_after)
    
    mult = dot /(x_len * y_len)
    if abs(mu - mult) > 0.00001:
        print(mu)
        print(mult)
        print("error")
    #    assert(False)

    
    sum_sq = (particle.direction.tetta_z * particle.direction.tetta_z + 
    particle.direction.tetta_y * particle.direction.tetta_y + 
    particle.direction.tetta_x * particle.direction.tetta_x)
    
    
    if (sum_sq > 1.01):
        print("sum sq "+ str(sum_sq))
        

In [239]:

# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[13]:


def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs/virtual_cs < random_number:
        return True
    else:
        return False


# In[14]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    print("process_virtual_collisions")



def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        set_random_direction(particle)
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        set_random_direction_anisotropic(particle, c_s, 1/3)
        
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        set_random_direction(particle)
        particle.set_terminated()
        
    return weight_before_collision


# In[16]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[17]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    delta_xs = universe.get_delta_xs()
    
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                print("reflect")
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[18]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[19]:


def make_sources(particles, c_s):
    for i in range(0, len(particles)):
        particles[i].terminated = False 
   #     set_random_direction(particles[i])
       # set_random_direction_anisotropic(particles[i], c_s, 1/3)


# In[20]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[21]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            



def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[23]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[24]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[25]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   


# In[26]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))



In [240]:

# In[27]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    radius = 0.77032
    diameter = 2 * 0.77032
    box_size = [diameter, diameter, diameter]
    zero_point = [-radius, -radius, -radius]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources, c_s)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [241]:
cs_fission_u235 = [0.266667]

cs_capture_u235 = [0.0]

cs_scattering_u235 = [0.733333]

cs_total_u235 = [1.]

cs_production_neutrons_u235 = [2.50]
cs_anisotropic =[[0.733333, 0.20]]


u235 = Sig(1, cs_fission_u235, cs_capture_u235, cs_scattering_u235, 
                       cs_production_neutrons_u235,  cs_total_u235, cs_anisotropic)


energy_groups_2 = [0, 2 * 1E10]

In [242]:
x_5 = Plane(0, 1, 0, 0.77032)
x_m5 = Plane(0, 1, 0, -0.77032)

y_5 = Plane(0, 0, 1, 100000)

y_m5 = Plane(0, 0, 1, -100000)


z_5 = Plane(1, 0, 0, 100000)
z_m5 = Plane(1, 0, 0, -100000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["black", "black", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000, -0.79606, 1000, 1000, 0.79606 )
slab.set_zero_point(0., 0., 0.)

In [243]:
cells = [slab]
estimators = []
materials = [u235]

U235_Reactor = Universe(cells, materials)

In [244]:

test_number_of_particles = 20
test_number_interations = 500
test_number_inactive = 50


# In[38]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, u235, estimators)


# In[40]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[42]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)


 num1  1
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 1.0202757888259655   0.9937424761740496  0.02653331265191583
i == 53
 keff_cycle , k_effective_exp, std_k_effective 0.970861100946903   0.9861153510983341  0.017112722275606753
i == 54
 keff_cycle , k_effective_exp, std_k_effective 0.9942822409423514   0.9881570735593385  0.012271563162400844
i == 55
 keff_cycle , k_effective_exp, std_k_effective 0.9948650509754389   0.9894986690425587  0.009599720616240483
i == 56
 keff_cycle , k_effective_exp, std_k_effective 0.9966253655488043   0.9906864517935996  0.007927625861730955
i 

 keff_cycle , k_effective_exp, std_k_effective 0.9835292418681816   0.9775115957924682  0.0019333277089317205
i == 120
 keff_cycle , k_effective_exp, std_k_effective 0.9894180063557408   0.9776816873719434  0.0019130849814107827
i == 121
 keff_cycle , k_effective_exp, std_k_effective 0.9919980696813923   0.9778833265594004  0.0018966963115555804
i == 122
 keff_cycle , k_effective_exp, std_k_effective 0.9801862926167213   0.9779153121990853  0.001870441290479165
i == 123
 keff_cycle , k_effective_exp, std_k_effective 0.9681461569332589   0.977781488154348  0.0018494887926479798
i == 124
 keff_cycle , k_effective_exp, std_k_effective 0.9899569935195014   0.9779460220106339  0.0018317290459199812
i == 125
 keff_cycle , k_effective_exp, std_k_effective 0.9974714313571137   0.9782063608019202  0.0018257970175545076
i == 126
 keff_cycle , k_effective_exp, std_k_effective 0.9697262261087239   0.9780947800822728  0.001805065208695207
i == 127
 keff_cycle , k_effective_exp, std_k_effective 0.96

 keff_cycle , k_effective_exp, std_k_effective 0.9813775123852542   0.9772388163030219  0.0013535633569196535
i == 190
 keff_cycle , k_effective_exp, std_k_effective 0.9825463443018979   0.9772767272172996  0.0013443949044723872
i == 191
 keff_cycle , k_effective_exp, std_k_effective 0.9597708231984337   0.9771525718696481  0.0013405876932496472
i == 192
 keff_cycle , k_effective_exp, std_k_effective 0.966442910055886   0.97707715171603  0.0013332483766364477
i == 193
 keff_cycle , k_effective_exp, std_k_effective 0.9722812237162562   0.9770436137579898  0.0013243168699432946
i == 194
 keff_cycle , k_effective_exp, std_k_effective 0.9851691126935673   0.9771000408339313  0.0013162980828292396
i == 195
 keff_cycle , k_effective_exp, std_k_effective 0.9553221425806346   0.9769498484321844  0.001315788702177052
i == 196
 keff_cycle , k_effective_exp, std_k_effective 0.9739704899772372   0.9769294418674244  0.001306904701787984
i == 197
 keff_cycle , k_effective_exp, std_k_effective 0.9880

 keff_cycle , k_effective_exp, std_k_effective 0.9818712230679596   0.9758094277994214  0.0010812879942617396
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.9856752108466104   0.9758564077186936  0.0010771516867366254
i == 261
 keff_cycle , k_effective_exp, std_k_effective 0.972733782314451   0.9758416085461618  0.001072136692243075
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.0004629634219282   0.975957747012557  0.001073369045002532
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.9633592993287621   0.9758985993708491  0.0010699539764532132
i == 264
 keff_cycle , k_effective_exp, std_k_effective 0.9865347162342568   0.9759483008515192  0.0010661016224356786
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.9431255285597294   0.975795636794348  0.0010720570016070176
i == 266
 keff_cycle , k_effective_exp, std_k_effective 0.9550922011303904   0.9756997875551631  0.001071378349201082
i == 267
 keff_cycle , k_effective_exp, std_k_effective 0.97530

 keff_cycle , k_effective_exp, std_k_effective 0.9770328979282829   0.9762573288996717  0.0009306243796839535
i == 330
 keff_cycle , k_effective_exp, std_k_effective 0.9671806378672504   0.9762249121459845  0.000927861210916899
i == 331
 keff_cycle , k_effective_exp, std_k_effective 0.9581045572866702   0.9761604268973749  0.000926799430182512
i == 332
 keff_cycle , k_effective_exp, std_k_effective 0.979676460397284   0.9761728951012753  0.0009235912217938506
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9917388428652297   0.9762278984502646  0.0009219640438844991
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9432750020288079   0.976111867124837  0.0009260102194728082
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9940348352579006   0.9761747547323213  0.0009248958154768442
i == 336
 keff_cycle , k_effective_exp, std_k_effective 0.9785379310220997   0.9761830175864814  0.0009216932789181461
i == 337
 keff_cycle , k_effective_exp, std_k_effective 0.958

 keff_cycle , k_effective_exp, std_k_effective 0.9971886821021367   0.9760790651940018  0.0008298020725941975
i == 400
 keff_cycle , k_effective_exp, std_k_effective 0.978753234792448   0.9760867056785688  0.0008274630883105511
i == 401
 keff_cycle , k_effective_exp, std_k_effective 0.9640812535080677   0.9760525021111315  0.0008258109042658974
i == 402
 keff_cycle , k_effective_exp, std_k_effective 0.935234405696733   0.9759365416099542  0.000831586251674522
i == 403
 keff_cycle , k_effective_exp, std_k_effective 0.9703749379160538   0.9759207863586968  0.0008293767977575844
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9516526396776291   0.9758522322720271  0.0008298670300654267
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.9683793458719839   0.9758311818878016  0.0008277937683718621
i == 406
 keff_cycle , k_effective_exp, std_k_effective 0.9871397673822424   0.9758629475773928  0.0008260762101012431
i == 407
 keff_cycle , k_effective_exp, std_k_effective 0.95

 keff_cycle , k_effective_exp, std_k_effective 0.9875945755981924   0.9754008492805699  0.0007662493137872367
i == 469
 keff_cycle , k_effective_exp, std_k_effective 0.973415814104775   0.975396111726451  0.0007644330495389487
i == 470
 keff_cycle , k_effective_exp, std_k_effective 0.9361228062787201   0.9753026038563374  0.0007683221670593415
i == 471
 keff_cycle , k_effective_exp, std_k_effective 0.9619754354999275   0.9752709478744932  0.0007671484132889229
i == 472
 keff_cycle , k_effective_exp, std_k_effective 0.9765430350905607   0.9752739622991758  0.0007653343035247472
i == 473
 keff_cycle , k_effective_exp, std_k_effective 0.9747648304922482   0.9752727586778829  0.000763523807543024
i == 474
 keff_cycle , k_effective_exp, std_k_effective 0.9664853559665038   0.975252033671488  0.0007620028075185605
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.9743423385724282   0.9752498932124315  0.0007602107587658179
i == 476
 keff_cycle , k_effective_exp, std_k_effective 0.954

In [245]:

difference = (k_effective_exp[-1] - 1.) * 100000
k_effective_std = get_std(k_effective)
standart_deviation = k_effective_std * 100000
print("k effective ")
print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))

k effective 
 difference from beachmark [pcm]  -2478.771571827887  with standart deviation [pcm]+- 73.52447548782996


In [895]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)